In [51]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [52]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Albany,42.6001,-73.9662,12.00,46,85,1.99,US,2021-02-13 14:10:05
1,1,Rikitea,-23.1203,-134.9692,79.14,77,0,14.74,PF,2021-02-13 14:11:04
2,2,Saint-Philippe,-21.3585,55.7679,84.20,65,0,27.63,RE,2021-02-13 14:11:04
3,3,Tessalit,20.1986,1.0114,92.48,9,0,5.82,ML,2021-02-13 14:11:05
4,4,Hilo,19.7297,-155.0900,66.20,82,40,3.49,US,2021-02-13 14:11:05


In [53]:
city_data_df.dtypes

City_ID         int64
City           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
Country        object
Date           object
dtype: object

In [54]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [55]:
%matplotlib inline

In [56]:
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))

In [57]:
locations = city_data_df[["Lat", "Lng"]]
max_temp = city_data_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp], dissipating=False, max_intensity=300, point_radius=4)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [58]:
print(fig)

Figure(layout=FigureLayout(height='420px'))


In [59]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [60]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))


What is the minimum temperature you would like for your trip? 50
What is the maximum temperature you would like for your trip? 80


In [61]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
1,1,Rikitea,-23.1203,-134.9692,79.14,77,0,14.74,PF,2021-02-13 14:11:04
4,4,Hilo,19.7297,-155.0900,66.20,82,40,3.49,US,2021-02-13 14:11:05
7,7,Ixtapa,20.7000,-105.2000,66.00,88,90,3.44,MX,2021-02-13 14:11:05
9,9,Airai,-8.9266,125.4092,62.04,100,69,0.49,TL,2021-02-13 14:11:05
10,10,Vaini,-21.2000,-175.2000,78.80,89,20,5.75,TO,2021-02-13 14:11:05
11,11,Henties Bay,-22.1160,14.2845,70.52,83,76,7.11,NaN,2021-02-13 14:11:05
12,12,Tabas,33.5959,56.9244,75.20,12,0,2.30,IR,2021-02-13 14:11:05
13,13,Cape Town,-33.9258,18.4232,77.00,65,0,19.57,ZA,2021-02-13 14:08:09
17,17,Waingapu,-9.6567,120.2641,79.14,82,100,5.82,ID,2021-02-13 14:11:06
19,19,Hermanus,-34.4187,19.2345,78.01,71,0,11.01,ZA,2021-02-13 14:11:06


In [62]:
preferred_cities_df.count()

City_ID       210
City          210
Lat           210
Lng           210
Max Temp      210
Humidity      210
Cloudiness    210
Wind Speed    210
Country       206
Date          210
dtype: int64

In [63]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
1,Rikitea,PF,79.14,-23.1203,-134.9692,
4,Hilo,US,66.20,19.7297,-155.0900,
7,Ixtapa,MX,66.00,20.7000,-105.2000,
9,Airai,TL,62.04,-8.9266,125.4092,
10,Vaini,TO,78.80,-21.2000,-175.2000,
11,Henties Bay,NaN,70.52,-22.1160,14.2845,
12,Tabas,IR,75.20,33.5959,56.9244,
13,Cape Town,ZA,77.00,-33.9258,18.4232,
17,Waingapu,ID,79.14,-9.6567,120.2641,
19,Hermanus,ZA,78.01,-34.4187,19.2345,


In [64]:
# Dependencies and Setup
import requests
import gmaps

# Import API key
from config import g_key


In [65]:
# Set the parameters to search for a hotel in Paris.
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key,
    "location": "48.8566, 2.3522"}
# Use base URL to search for hotels in Paris.
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
# Make request and get the JSON data from the search.
hotels = requests.get(base_url, params=params).json()

hotels

{'html_attributions': [],
 'next_page_token': 'ATtYBwLQiZIQEf7zpLU2Btvcw9JvPbB5Mbfb3X31GhfLDMZKNABvjt9NhlPtrym9mf_HMHzACTXueGp2m2unC7yd4URFS9WroC19mmGpA8N9XZbhwdlZVYx736jpdo8G6H8IWudE61BWpX5aoEphQ_gGvUpAnyQyEWYTpxgBNxuaUSWpfaFgn7Y1Cj9dzdWmslnku4geP-PTpAELoGPE2FSI_D7DHcdwvSR54t5b6Vyt0_5XpIedJjJa34pUqyDN36hqORSxRd5cUiBf0Zz-qmVfSxQygja3S9YL5XVtKDXYNEOxOz1HFWSIIaR9JEwA3OAvaVNdTH7FOyHR5AlJ4qupCi3BZWKAgMRSyCLQWKi3xeRt_zjOTJ7l84iHBX63Uhn7wEmamxeuC1OXUbNcxVAtJzGFafDCguqSqugiFaUD4kRTfiIXrCT9uGlb',
 'results': [{'business_status': 'OPERATIONAL',
   'geometry': {'location': {'lat': 48.8581126, 'lng': 2.3529277},
    'viewport': {'northeast': {'lat': 48.8594393802915,
      'lng': 2.354352980291503},
     'southwest': {'lat': 48.8567414197085, 'lng': 2.351655019708499}}},
   'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png',
   'name': 'Hôtel Duo',
   'opening_hours': {'open_now': True},
   'photos': [{'height': 3840,
     'html_attributions': ['<a href="https://

In [66]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()

   # Grab the first hotel from the results and store the name.
    try:
     hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
     print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [50]:
# Add a heatmap of temperature for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [67]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
1,Rikitea,PF,79.14,-23.1203,-134.9692,
4,Hilo,US,66.20,19.7297,-155.0900,
7,Ixtapa,MX,66.00,20.7000,-105.2000,
9,Airai,TL,62.04,-8.9266,125.4092,
10,Vaini,TO,78.80,-21.2000,-175.2000,
11,Henties Bay,NaN,70.52,-22.1160,14.2845,
12,Tabas,IR,75.20,33.5959,56.9244,
13,Cape Town,ZA,77.00,-33.9258,18.4232,
17,Waingapu,ID,79.14,-9.6567,120.2641,
19,Hermanus,ZA,78.01,-34.4187,19.2345,


In [68]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [76]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
            hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")


Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [77]:
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
1,Rikitea,PF,79.14,-23.1203,-134.9692,Pension Maro'i
4,Hilo,US,66.20,19.7297,-155.0900,Hilo Hawaiian Hotel
7,Ixtapa,MX,66.00,20.7000,-105.2000,Hotel Plaza Providencia
9,Airai,TL,62.04,-8.9266,125.4092,Juvinal Martin
10,Vaini,TO,78.80,-21.2000,-175.2000,Keleti Beach Resort
11,Henties Bay,NaN,70.52,-22.1160,14.2845,Fishermans Guest House
12,Tabas,IR,75.20,33.5959,56.9244,شرکت معماری مهراز شایگان آراد
13,Cape Town,ZA,77.00,-33.9258,18.4232,Southern Sun Waterfront Cape Town
17,Waingapu,ID,79.14,-9.6567,120.2641,Pondok Wisata Elim
19,Hermanus,ZA,78.01,-34.4187,19.2345,Misty Waves Boutique Hotel


In [78]:
# Add a heatmap of temperature for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [79]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [81]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]


In [82]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))